<a href="https://colab.research.google.com/github/Nan2711/NEMO/blob/main/Nemo_English_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

# ## Install NeMo
BRANCH = 'r1.5.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.5.0) to /tmp/pip-install-9o18etf_/nemo-toolkit_f3adb14f2f114ee1a8a61eefce196f18
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-9o18etf_/nemo-toolkit_f3adb14f2f114ee1a8a61eefce196f18
  Running command git checkout -b r1.5.0 --track origin/r1.5.0
  Switched to a new branch 'r1.5.0'
  Branch 'r1.5.0' set up to track remote branch 'r1.5.0' from 'origin'.
  Using cached nltk-3.6.7-py3-none-any.whl (1.5 MB)
mkdir: cannot create directory ‘configs’: File exists


In [ ]:
import os
DATA_DIR = "DATA_DIR"
WORK_DIR = "WORK_DIR"
#os.environ['DATA_DIR'] = DATA_DIR

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

! unzip -o dataset.zip -d {DATA_DIR}


Archive:  dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of dataset.zip or
        dataset.zip.zip, and cannot find dataset.zip.ZIP, period.


In [ ]:
import nemo
import nemo.collections.nlp as nemo_nlp
nemo.__version__

[NeMo W 2022-01-23 04:35:12 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-01-23 04:35:15 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.decoder_dataset.TextNormalizationDecoderDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-23 04:35:15 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.tagger_dataset.TextNormalizationTaggerDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-23 04:35:15 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.test_dataset.TextNormalizationTestDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-23 04:35:15 __init__:23] `pynini` is not installed ! 
    Please run the `nemo_text_processing/setup.sh` scriptprior to usage of this

'1.5.0'

In [ ]:
nlp_models = [model for model in dir(nemo_nlp.models) if model.endswith("Model")]
nlp_models

['BERTLMModel',
 'BertDPRModel',
 'BertJointIRModel',
 'DuplexDecoderModel',
 'DuplexTaggerModel',
 'DuplexTextNormalizationModel',
 'EntityLinkingModel',
 'GLUEModel',
 'IntentSlotClassificationModel',
 'MTEncDecModel',
 'PunctuationCapitalizationModel',
 'QAModel',
 'SGDQAModel',
 'Text2SparqlModel',
 'TextClassificationModel',
 'TokenClassificationModel',
 'TransformerLMModel',
 'ZeroShotIntentModel']

In [ ]:
from nemo.collections.nlp.models import MTEncDecModel
MTEncDecModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=nmt_en_de_transformer12x2,
 	description=En->De translation model. See details here: https://ngc.nvidia.com/catalog/models/nvidia:nemo:nmt_en_de_transformer12x2,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/nmt_en_de_transformer12x2/versions/1.0.0rc1/files/nmt_en_de_transformer12x2.nemo
 ), PretrainedModelInfo(
 	pretrained_model_name=nmt_de_en_transformer12x2,
 	description=De->En translation model. See details here: https://ngc.nvidia.com/catalog/models/nvidia:nemo:nmt_de_en_transformer12x2,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/nmt_de_en_transformer12x2/versions/1.0.0rc1/files/nmt_de_en_transformer12x2.nemo
 ), PretrainedModelInfo(
 	pretrained_model_name=nmt_en_es_transformer12x2,
 	description=En->Es translation model. See details here: https://ngc.nvidia.com/catalog/models/nvidia:nemo:nmt_en_es_transformer12x2,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/nmt_en_es_transformer12x2/ve

In [ ]:
-

nmt_model = nemo_nlp.models.machine_translation.MTEncDecModel.from_pretrained(model_name="nmt_en_hi_transformer12x2")

[NeMo I 2022-01-23 04:35:17 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/nmt_en_hi_transformer12x2/versions/v1.0.0/files/nmt_en_hi_transformer12x2.nemo to /root/.cache/torch/NeMo/NeMo_1.5.0/nmt_en_hi_transformer12x2/840c0b573558ff018a051565880ff149/nmt_en_hi_transformer12x2.nemo
[NeMo I 2022-01-23 04:35:52 common:728] Instantiating model from pre-trained checkpoint
[NeMo I 2022-01-23 04:36:05 tokenizer_utils:163] Getting YouTokenToMeTokenizer with model: /tmp/tmpdez5ie8z/11dfc6ccf23343bcb2b5ac9c544c4b56_shared_tokenizer.32000.BPE.model with r2l: False.
[NeMo I 2022-01-23 04:36:05 tokenizer_utils:163] Getting YouTokenToMeTokenizer with model: /tmp/tmpdez5ie8z/0defb13ccbe942d88d7b1c31d8c96173_shared_tokenizer.32000.BPE.model with r2l: False.


[NeMo W 2022-01-23 04:36:05 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    src_file_name: null
    tgt_file_name: null
    use_tarred_dataset: true
    tar_files: /raid/data/preproc/parallel.batches.tokens.16000._OP_0..57_CL_.tar
    metadata_file: /raid/data/preproc/metadata.tokens.16000.json
    lines_per_dataset_fragment: 1000000
    num_batches_per_tarfile: 100
    shard_strategy: scatter
    tokens_in_batch: 16000
    clean: true
    max_seq_length: 512
    min_seq_length: 1
    cache_ids: false
    cache_data_per_node: false
    use_cache: false
    shuffle: true
    num_samples: -1
    drop_last: false
    pin_memory: false
    num_workers: 8
    reverse_lang_direction: false
    load_from_tarred_dataset: false
    metadata_path: null
    tar_shuffle_n: 100
    n_preproc_jobs: -2
    tar_file_prefix: parallel
    concat_sa

[NeMo I 2022-01-23 04:36:11 save_restore_connector:149] Model MTEncDecModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.5.0/nmt_en_hi_transformer12x2/840c0b573558ff018a051565880ff149/nmt_en_hi_transformer12x2.nemo.


In [ ]:
translations = nmt_model.translate(["How are You?"], source_lang="en", target_lang="hi")

[NeMo W 2022-01-23 04:36:12 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/nlp/modules/common/transformer/transformer_generators.py:363: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      mems_ids = indices_i.unsqueeze(2).unsqueeze(3).repeat(1, 1, p_len - 1, hidden_size) // self.beam_size
    


In [ ]:
translations

['आप कैसे हैं?']

In [ ]:
nmt_model.summarize()

[NeMo W 2022-01-23 04:36:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
      """Entry point for launching an IPython kernel.
    
[NeMo W 2022-01-23 04:36:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/model_summary.py:472: LightningDeprecationWarning: Argument `mode` in `LightningModule.summarize` is deprecated in v1.4 and will be removed in v1.6. Use `max_depth=1` to replicate `mode=top` behavior.
      "Argument `mode` in `LightningModule.summarize` is deprecated in v1.4"
    


  | Name         | Type                     | Params
----------------------------------------------------------
0 | encoder      | TransformerEncoderNM     | 183 M 
1 | decoder      | TransformerDecoderNM     | 66.4 M
2 | log_softmax  | TokenClassifier          | 32.8 M
3 | loss_fn      | SmoothedCrossEntropyLoss | 0     
4 | eval_loss_fn | NLLLoss                  | 0     
----------------------------------------------------------
250 M     Trainable params
0         Non-trainable params
250 M     Total params
1,001.297 Total estimated model params size (MB)

In [ ]:
from omegaconf import OmegaConf

In [ ]:
import copy

In [ ]:
cfg = copy.deepcopy(nmt_model.cfg)
print(OmegaConf.to_yaml(cfg))

train_ds:
  src_file_name: null
  tgt_file_name: null
  use_tarred_dataset: true
  tar_files: /raid/data/preproc/parallel.batches.tokens.16000._OP_0..57_CL_.tar
  metadata_file: /raid/data/preproc/metadata.tokens.16000.json
  lines_per_dataset_fragment: 1000000
  num_batches_per_tarfile: 100
  shard_strategy: scatter
  tokens_in_batch: 16000
  clean: true
  max_seq_length: 512
  min_seq_length: 1
  cache_ids: false
  cache_data_per_node: false
  use_cache: false
  shuffle: true
  num_samples: -1
  drop_last: false
  pin_memory: false
  num_workers: 8
  reverse_lang_direction: false
  load_from_tarred_dataset: false
  metadata_path: null
  tar_shuffle_n: 100
  n_preproc_jobs: -2
  tar_file_prefix: parallel
  concat_sampling_technique: temperature
  concat_sampling_temperature: 5
  concat_sampling_probabilities: null
validation_ds:
  src_file_name: /raid/data/val_split.filter.en
  tgt_file_name: /raid/data/val_split.filter.hi
  use_tarred_dataset: false
  tar_files: null
  metadata_file:

In [ ]:
print(OmegaConf.to_yaml(nmt_model.cfg))

train_ds:
  src_file_name: null
  tgt_file_name: null
  use_tarred_dataset: true
  tar_files: /raid/data/preproc/parallel.batches.tokens.16000._OP_0..57_CL_.tar
  metadata_file: /raid/data/preproc/metadata.tokens.16000.json
  lines_per_dataset_fragment: 1000000
  num_batches_per_tarfile: 100
  shard_strategy: scatter
  tokens_in_batch: 16000
  clean: true
  max_seq_length: 512
  min_seq_length: 1
  cache_ids: false
  cache_data_per_node: false
  use_cache: false
  shuffle: true
  num_samples: -1
  drop_last: false
  pin_memory: false
  num_workers: 8
  reverse_lang_direction: false
  load_from_tarred_dataset: false
  metadata_path: null
  tar_shuffle_n: 100
  n_preproc_jobs: -2
  tar_file_prefix: parallel
  concat_sampling_technique: temperature
  concat_sampling_temperature: 5
  concat_sampling_probabilities: null
validation_ds:
  src_file_name: /raid/data/val_split.filter.en
  tgt_file_name: /raid/data/val_split.filter.hi
  use_tarred_dataset: false
  tar_files: null
  metadata_file:

In [ ]:
!tar -xzvf "/content/parallel.tgz" -C "/content/DATA_DIR"     #[run this cell to extract tar.gz files]

parallel/
parallel/IITB.en-hi.hi

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
nmt_model.cfg.train_ds.src_file_name="/content/IITB.en-hi.en"
nmt_model.cfg.train_ds.tgt_file_name="/content/IITB.en-hi.hi"


In [ ]:
print(OmegaConf.to_yaml(nmt_model.cfg))

train_ds:
  src_file_name: /content/IITB.en-hi.en
  tgt_file_name: /content/IITB.en-hi.hi
  use_tarred_dataset: true
  tar_files: /raid/data/preproc/parallel.batches.tokens.16000._OP_0..57_CL_.tar
  metadata_file: /raid/data/preproc/metadata.tokens.16000.json
  lines_per_dataset_fragment: 1000000
  num_batches_per_tarfile: 100
  shard_strategy: scatter
  tokens_in_batch: 16000
  clean: true
  max_seq_length: 512
  min_seq_length: 1
  cache_ids: false
  cache_data_per_node: false
  use_cache: false
  shuffle: true
  num_samples: -1
  drop_last: false
  pin_memory: false
  num_workers: 8
  reverse_lang_direction: false
  load_from_tarred_dataset: false
  metadata_path: null
  tar_shuffle_n: 100
  n_preproc_jobs: -2
  tar_file_prefix: parallel
  concat_sampling_technique: temperature
  concat_sampling_temperature: 5
  concat_sampling_probabilities: null
validation_ds:
  src_file_name: /raid/data/val_split.filter.en
  tgt_file_name: /raid/data/val_split.filter.hi
  use_tarred_dataset: fals

In [ ]:
nmt_model.setup_training_data(cfg.train_ds)

FileNotFoundError: ignored

In [ ]:
# download the model's configuration file 
MODEL_CONFIG = "aayn_base.yaml"
CONFIG_DIR = WORK_DIR + '/configs/'

os.makedirs(CONFIG_DIR, exist_ok=True)
if not os.path.exists(CONFIG_DIR + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/machine_translation/conf/' + MODEL_CONFIG, CONFIG_DIR)
    print('Config file downloaded!')
else:
    print ('config file already exists')
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)

NameError: ignored

In [ ]:
print(OmegaConf.to_yaml(nmt_model.cfg))

train_ds:
  src_file_name: /content/IITB.en-hi.en
  tgt_file_name: /content/IITB.en-hi.hi
  use_tarred_dataset: true
  tar_files: /raid/data/preproc/parallel.batches.tokens.16000._OP_0..57_CL_.tar
  metadata_file: /raid/data/preproc/metadata.tokens.16000.json
  lines_per_dataset_fragment: 1000000
  num_batches_per_tarfile: 100
  shard_strategy: scatter
  tokens_in_batch: 16000
  clean: true
  max_seq_length: 512
  min_seq_length: 1
  cache_ids: false
  cache_data_per_node: false
  use_cache: false
  shuffle: true
  num_samples: -1
  drop_last: false
  pin_memory: false
  num_workers: 8
  reverse_lang_direction: false
  load_from_tarred_dataset: false
  metadata_path: null
  tar_shuffle_n: 100
  n_preproc_jobs: -2
  tar_file_prefix: parallel
  concat_sampling_technique: temperature
  concat_sampling_temperature: 5
  concat_sampling_probabilities: null
validation_ds:
  src_file_name: /raid/data/val_split.filter.en
  tgt_file_name: /raid/data/val_split.filter.hi
  use_tarred_dataset: fals

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(**config.trainer)

NameError: ignored